# <center>Data Mining Project</center>

<center>
Master in Data Science and Advanced Analytics <br>
NOVA Information Management School
</center>

** **
## <center>*ABCDEats Inc*</center>

<center>
Group 19 <br>
Jan-Louis Schneider, 20240506  <br>
Marta Boavida, 20240519  <br>
Matilde Miguel, 20240549  <br>
Sofia Gomes, 20240848  <br>
</center>

** **

In [ ]:
!jupyter nbconvert --to script "New_Features.ipynb"

In [ ]:
import pandas as pd 
import numpy as np
import scipy

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency
import scipy.stats as stats
import warnings

from math import ceil
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.decomposition import PCA
from collections import Counter

import plotly.express as px

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

from New_Features import * 

from sklearn.feature_selection import VarianceThreshold

In [ ]:
df = pd.read_csv("../dataset/df_transform.csv")

In [ ]:
df.head(20)

## <span style="color:salmon">6. Clustering </span> 


#### <span style="color:salmon"> 6.1 Subtítulo </span> 